In [1]:
import h5py, os, pickle
import numpy as np
from imp import reload

from ecog.utils import bands
from ecog.signal_processing import resample

from cv_paper_plots.broadband import forward_bl, baseline_mean_std, load_data

/home/jesse/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# Load data

In [2]:
folder = os.path.join(os.environ['HOME'],'Development/data/ecog/AA_ff/')
subjects = ['ec2', 'ec9', 'gp31', 'gp33']
files = ['EC2_blocks_1_8_9_15_76_89_105_CV_AA_ff_align_window_-0.5_to_0.79_none.h5',
         'EC9_blocks_15_39_46_49_53_60_63_CV_AA_ff_align_window_-0.5_to_0.79_none.h5',
         'GP31_blocks_1_2_4_6_9_21_63_65_67_69_71_78_82_83_CV_AA_ff_align_window_-0.5_to_0.79_none.h5',
         'GP33_blocks_1_5_30_CV_AA_ff_align_window_-0.5_to_0.79_none.h5']
cfs = bands.chang_lab['cfs']

# No 1f

In [ ]:
for subject_idx in range(4):
    subject = subjects[subject_idx]
    fname = os.path.join(folder, files[subject_idx])
    X, baselines, good_examples, good_channels, tokens, block_labels, labels = load_data(fname)
    with open('{}_bl_bb.pkl'.format(subject), 'rb') as f:
        bl_bb = pickle.load(f)
    bl_mean_bb, bl_std_bb = baseline_mean_std(block_labels, good_channels, baselines)
    X, _ = forward_bl(X, 'bl_zscore', bl_mean_bb, bl_std_bb, block_labels)
    HG_freq = bands.neuro['HG_freq']
    new_fname = os.path.splitext(fname)[0]
    new_fname = '{}_AA_avg.h5'.format(new_fname)
    min_freqs = bands.neuro['min_freqs']
    max_freqs = bands.neuro['max_freqs']
    with h5py.File(new_fname, 'w') as f:
        f.create_dataset('y', data=labels)
        for ii, b in enumerate(bands.neuro['bands']):
            minf = min_freqs[ii]
            maxf = max_freqs[ii]
            target_fs = (HG_freq * (minf + maxf) /
                                 (max_freqs[-1] + min_freqs[-1]))
            idxs = np.where(np.logical_and(cfs >= minf, cfs <= maxf))[0]
            if not np.allclose(target_fs, HG_freq):
                Xp = resample(X[idxs], target_fs, HG_freq).mean(axis=0)
            else:
                Xp = X[idxs].mean(axis=0)
            f.create_dataset('X{}'.format(b), data=Xp.astype('float32'))

2572 86


# 1f

In [3]:
for subject_idx in range(4):
    subject = subjects[subject_idx]
    fname = os.path.join(folder, files[subject_idx])
    X, baselines, good_examples, good_channels, tokens, block_labels, labels = load_data(fname)
    with open('{}_bl_bb.pkl'.format(subject), 'rb') as f:
        bl_bb = pickle.load(f)
    bl_mean_bb, bl_std_bb = baseline_mean_std(block_labels, good_channels, baselines, mb=bl_bb, cfs=cfs)
    d = np.load('{}_bb.npz'.format(subject))
    X_fits = d['X_bls']
    X_medR2 = d['X_medR2']
    kind = d['kind']
    X, _ = forward_bl(X, 'bl_zscore_bb', bl_mean_bb, bl_std_bb, block_labels, X_fits, cfs)
    HG_freq = bands.neuro['HG_freq']
    new_fname = os.path.splitext(fname)[0]
    new_fname = '{}_AA_avg_1f.h5'.format(new_fname)
    min_freqs = bands.neuro['min_freqs']
    max_freqs = bands.neuro['max_freqs']
    with h5py.File(new_fname, 'w') as f:
        f.create_dataset('y', data=labels)
        for ii, b in enumerate(bands.neuro['bands']):
            minf = min_freqs[ii]
            maxf = max_freqs[ii]
            target_fs = (HG_freq * (minf + maxf) /
                                 (max_freqs[-1] + min_freqs[-1]))
            idxs = np.where(np.logical_and(cfs >= minf, cfs <= maxf))[0]
            if not np.allclose(target_fs, HG_freq):
                Xp = resample(X[idxs], target_fs, HG_freq).mean(axis=0)
            else:
                Xp = X[idxs].mean(axis=0)
            f.create_dataset('X{}'.format(b), data=Xp.astype('float32'))

2572 85
1570 77


TypeError: 'dict' object is not callable